In [248]:
import pandas as pd
import gspread
import requests

token = 'AF5y90PW7vmJALond4SEGVBFwmt3Jhj7Ba7oclAeXL4'

In [249]:
sa = gspread.service_account(filename='geprek-wani-abcd00886732.json')

In [250]:
sheet = sa.open('Wani Spreadsheet')

In [251]:
database = sheet.worksheet('Database Harga')

In [252]:
df_harga = pd.DataFrame(database.get_all_records())

In [253]:
worksheet = sheet.worksheet('Pesanan')

In [254]:
df = pd.DataFrame(worksheet.get_all_records())

In [255]:
df.rename(columns={'Request cabe GEPREK atau cabe TERIYAKI': 'Cabe', 
                   'Sambal Tambahan (Rp 2.000,-)': 'Sambal',
                   'Camilan (Rp 10.000,-)': 'Camilan',
                   }, inplace=True)

In [256]:
df.head()

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal,"Bukti Pembayaran, transfer ke BCA: 1500952195 a/n BILLY JASON MARGIANTO","Tambah Nasi Putih (Rp 5.000,-)"
0,25/05/2023 08:26:34,Tot,"Nasi Ayam Crisbar ~ Rp 15.000,-",,,"Es Teh Jumbo ~ Rp 5.000,-",,https://drive.google.com/open?id=1VIZqrJIl3tNb...,
1,25/05/2023 08:27:04,Tot,"Ayam Geprek tanpa Nasi ~ Rp 9.000,-",1,,,,https://drive.google.com/open?id=1IBTKeEDK-sWY...,
2,25/05/2023 08:27:18,Fathullah Auzan,"Nasi Ayam Geprek ~ Rp 12.000,-",5,,,,https://drive.google.com/open?id=1x-7njIiEcXZx...,
3,25/05/2023 08:27:28,Irfan dary,"Nasi Ayam Geprek ~ Rp 12.000,-",3,,,,https://drive.google.com/open?id=1ebNLP0meJMzS...,
4,25/05/2023 08:27:40,Max,"Nasi Ayam Geprek ~ Rp 12.000,-",,,,,https://drive.google.com/open?id=1KWMRGrIesJbV...,


In [257]:
# create a list containing 'Makanan Utama' and 'Nama'
list = ['Makanan Utama', 'Cabe', 'Nama']

# replace the empty string with 1 if Makanan Utama is not empty
df['Cabe'] = df['Cabe'].apply(lambda x: 1 if x == '' else x)

# remove everything after ' ~' in Makanan Utama
df['Makanan Utama'] = df['Makanan Utama'].apply(lambda x: x.split(' ~')[0])
df['Minuman'] = df['Minuman'].apply(lambda x: x.split(' ~')[0])

df[list]

,Makanan Utama,Cabe,Nama
0,Nasi Ayam Crisbar,1,Tot
1,Ayam Geprek tanpa Nasi,1,Tot
2,Nasi Ayam Geprek,5,Fathullah Auzan
3,Nasi Ayam Geprek,3,Irfan dary
4,Nasi Ayam Geprek,1,Max
5,Nasi Udang Geprek,3,Aiffah
6,Nasi Ayam Teriyaki,1,fazaa
7,Nasi Babat Goreng,1,kevin
8,Nasi Ayam Bakar,1,Jalu
9,Nasi Ayam Crisbar,1,verill


In [258]:
# Custom geprek
df_geprek = df[df['Makanan Utama'].str.contains('Geprek', case=False) & ~df['Makanan Utama'].str.contains('Indomie') & ~df['Makanan Utama'].str.contains('Udang')]
df_geprek

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal,"Bukti Pembayaran, transfer ke BCA: 1500952195 a/n BILLY JASON MARGIANTO","Tambah Nasi Putih (Rp 5.000,-)"
1,25/05/2023 08:27:04,Tot,Ayam Geprek tanpa Nasi,1,,,,https://drive.google.com/open?id=1IBTKeEDK-sWY...,
2,25/05/2023 08:27:18,Fathullah Auzan,Nasi Ayam Geprek,5,,,,https://drive.google.com/open?id=1x-7njIiEcXZx...,
3,25/05/2023 08:27:28,Irfan dary,Nasi Ayam Geprek,3,,,,https://drive.google.com/open?id=1ebNLP0meJMzS...,
4,25/05/2023 08:27:40,Max,Nasi Ayam Geprek,1,,,,https://drive.google.com/open?id=1KWMRGrIesJbV...,


In [259]:
# Dataframe not-geprek but include 'Indomie' and 'Udang'

df_not_geprek = df[~df['Makanan Utama'].str.contains('geprek', case=False)]

# drop rows with empty Makanan Utama
df_not_geprek = df_not_geprek[df_not_geprek['Makanan Utama'] != '']

# Dataframe geprek tanpa nasi
df_geprek_tanpa_nasi = df_geprek[df_geprek['Makanan Utama'].str.contains('tanpa Nasi')]

# Dataframe geprek dengan nasi
df_geprek_dengan_nasi = df_geprek[~df_geprek['Makanan Utama'].str.contains('tanpa Nasi')]

# Dataframe indomie geprek
df_indomie_geprek = df[df['Makanan Utama'].str.contains('Indomie') & df['Makanan Utama'].str.contains('Geprek')]

# Dataframe udang geprek
df_udang_geprek = df[df['Makanan Utama'].str.contains('Udang') & df['Makanan Utama'].str.contains('Geprek')]

# Dataframe camilan
df_camilan = df[['Camilan', 'Nama']]
df_camilan = df_camilan[df_camilan['Camilan'] != '']

# Dataframe minuman
df_minuman = df[['Minuman', 'Nama']]
df_minuman = df_minuman[df_minuman['Minuman'] != '']

# Dataframe sambal
df_sambal = df[['Sambal', 'Nama']]
df_sambal = df_sambal[df_sambal['Sambal'] != '']

In [260]:
combined_prints = []
combined_prints.append(f"Pesanan Depot Wani: {pd.Timestamp.now()}")
combined_prints.append("=====================================")
combined_prints.extend([f"{sum(df_geprek_dengan_nasi['Cabe'] == i)} Nasi Ayam Geprek Cabe {i} ({', '.join(df_geprek_dengan_nasi[df_geprek_dengan_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_dengan_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_geprek_tanpa_nasi['Cabe'] == i)} Ayam Geprek tanpa nasi Cabe {i} ({', '.join(df_geprek_tanpa_nasi[df_geprek_tanpa_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_tanpa_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_not_geprek['Makanan Utama'] == i)} {i} ({', '.join(df_not_geprek[df_not_geprek['Makanan Utama'] == i]['Nama'])})" for i in df_not_geprek['Makanan Utama'].unique()])
combined_prints.extend([f"{sum(df_indomie_geprek['Cabe'] == i)} {df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Makanan Utama'].iloc[0]} Cabe {i} ({', '.join(df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Nama'])})" for i in df_indomie_geprek['Cabe'].unique()])
combined_prints.extend([f"{sum(df_udang_geprek['Cabe'] == i)} {df_udang_geprek[df_udang_geprek['Cabe'] == i]['Makanan Utama'].iloc[0]} Cabe {i} ({', '.join(df_udang_geprek[df_udang_geprek['Cabe'] == i]['Nama'])})" for i in df_udang_geprek['Cabe'].unique()])
combined_prints.extend([f"{sum(df_camilan['Camilan'] == i)} {i} ({', '.join(df_camilan[df_camilan['Camilan'] == i]['Nama'])})" for i in df_camilan['Camilan'].unique()])
combined_prints.extend([f"{sum(df_minuman['Minuman'] == i)} {i} ({', '.join(df_minuman[df_minuman['Minuman'] == i]['Nama'])})" for i in df_minuman['Minuman'].unique()])
combined_prints.extend([f"{sum(df_sambal['Sambal'] == i)} {i} ({', '.join(df_sambal[df_sambal['Sambal'] == i]['Nama'])})" for i in df_sambal['Sambal'].unique()])

print('\n'.join(combined_prints))

Pesanan Depot Wani: 2023-05-25 11:57:53.691859
1 Nasi Ayam Geprek Cabe 5 (Fathullah Auzan)
1 Nasi Ayam Geprek Cabe 3 (Irfan dary)
1 Nasi Ayam Geprek Cabe 1 (Max)
1 Ayam Geprek tanpa nasi Cabe 1 (Tot)
2 Nasi Ayam Crisbar (Tot, verill)
4 Nasi Ayam Teriyaki (fazaa, Kevin Sander Utomo, Awang, MW)
1 Nasi Babat Goreng (kevin)
2 Nasi Ayam Bakar (Jalu, Reyner Fernaldi)
1 Nasi Ayam Goreng Lengkuas (JoWi)
1 Indomie Ayam Geprek Cabe 10 (Galih)
1 Indomie Ayam Geprek Cabe 1 (Rafi)
1 Nasi Udang Geprek Cabe 3 (Aiffah)
1 Kulit Ayam Crispy (JoWi)
1 Tahu Crispy (Evan)
2 Es Teh Jumbo (Tot, Rafi)
1 Es Teh (Galih)
1 Sambal Bawang (verill)


In [261]:
# remove all text after '(' in combined_prints
# combined_prints = [i.split('(')[0] for i in combined_prints]

# print('\n'.join(combined_prints))

In [262]:
# payload = {'message' : '\n'.join(combined_prints)}
# r = requests.post('https://notify-api.line.me/api/notify'
#                 , headers={'Authorization' : 'Bearer {}'.format(token)}
#                 , params = payload)

In [263]:
# duar
# payload = {'message' : 'Duar'}
# files = {'imageFile': open('duar.jpg', 'rb')}
# r = requests.post('https://notify-api.line.me/api/notify'
#                 , headers={'Authorization' : 'Bearer {}'.format(token)}
#                 , params = payload, files=files)

In [264]:
# ambatukam
# payload = {'message' : 'Ambatukam @Jason'}
# files = {'imageFile': open('ambatukam.jpg', 'rb')}

# r = requests.post('https://notify-api.line.me/api/notify'
#                 , headers={'Authorization' : 'Bearer {}'.format(token)}
#                 , params = payload, files=files)